In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
!pip install xgboost
import xgboost as xgb

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Multi-modal learning
The way to approach this problem is to separately process the text and quantitative data. <br> For the text data, we will need to transform it into numerical form using an embedding of some sort. <br> The quantitative data can be pre-processed using the standard pre-processing techniques such as normalization. <br>The two sets of data are then concate to put into one single classifier ML algorithm.
<br><br> To tackle this challenge, we will utilize basic algorithms first before stepping up to transformers, GPT embeddings etc.

In [2]:
fomc_statements = pd.read_excel('FOMC Statements 1997-2023.xlsx')
fomc_statements.rename(columns={"Meeting Date":"Meeting_Date", "Unnamed: 1":"Statement"}, inplace=True)
fomc_statements

,Meeting_Date,Statement
0,1997-03-25,_x000D_\n_x000D_\n\tThe Federal Open Market Co...
1,1998-09-29,_x000D_\nThe Federal Open Market Committee dec...
2,1998-10-15,_x000D_\nThe Federal Reserve today announced t...
3,1998-11-17,_x000D_\nThe Federal Reserve today announced t...
4,1999-05-18,_x000D_\nThe Federal Reserve released the foll...
...,...,...
201,2022-09-21,\nRecent indicators point to modest growth in ...
202,2022-11-02,\nRecent indicators point to modest growth in ...
203,2022-12-14,\nRecent indicators point to modest growth in ...
204,2023-02-01,\nRecent indicators point to modest growth in ...


In [3]:
econs_metrics = pd.read_excel('FOMC_econometrics_v1.xlsx')
econs_metrics.rename(columns={'Dates':'Meeting_Date'}, inplace=True)
econs_metrics

,Meeting_Date,Label,Fed_fund_rate,Unemployment_rate,Core_CPI,US_10-2_Spread,Fed_fund_rate_delta,Unemployment_rate_delta,Core_CPI_delta,US_10-2_Spread_delta
0,1997-03-25,1,5.47,5.2,2.5,0.4530,0.43,-0.2,-0.1,-0.0530
1,1998-09-29,0,5.55,4.5,2.5,0.1200,-0.91,0.0,0.3,0.1340
2,1998-10-15,0,5.33,4.6,2.5,0.5820,0.07,0.1,0.3,0.5300
3,1998-11-17,0,5.45,4.5,2.3,0.3100,-0.87,0.0,0.1,0.2290
4,1999-05-18,0,5.01,4.3,2.2,0.3160,-0.09,0.0,-0.2,0.2140
...,...,...,...,...,...,...,...,...,...,...
201,2022-09-21,1,2.33,3.7,6.3,-0.5185,0.75,0.1,0.3,-0.5910
202,2022-11-02,1,3.08,3.7,6.3,-0.5192,0.75,0.2,0.4,-0.1648
203,2022-12-14,0,3.83,3.6,6.0,-0.7321,1.50,-0.1,-0.3,-0.3164
204,2023-02-01,0,4.33,3.4,5.6,-0.6897,0.50,-0.3,-0.7,-0.1228


In [4]:
combined_df = pd.merge(fomc_statements, econs_metrics, on='Meeting_Date') #Merging the dataframes, this is to ensure that the metrics matches the statements and labels.
y_labels = combined_df['Label']
x_text = combined_df['Statement']
x_metrics = combined_df[['Fed_fund_rate', 'Unemployment_rate', 'Core_CPI', 'US_10-2_Spread']]
x_metrics_delta = combined_df[['Fed_fund_rate_delta', 'Unemployment_rate_delta', 'Core_CPI_delta', 'US_10-2_Spread_delta']]
x_metrics_all = combined_df[['Fed_fund_rate', 'Unemployment_rate', 'Core_CPI', 'US_10-2_Spread', 'Fed_fund_rate_delta', 'Unemployment_rate_delta', 'Core_CPI_delta', 'US_10-2_Spread_delta']]

# We will first utilize the simple TfidfVectorizer

In [33]:
### Initialize only for absolute metrics ###
DT_model_tfidf = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_tfidf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_tfidf = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

### Initialize only for metrics delta ###
DT_model_tfidf_delta = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_tfidf_delta = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_tfidf_delta = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

### Initialize only for all metrics ###
DT_model_tfidf_all = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_tfidf_all = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_tfidf_all = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

In [34]:
### Processing of text data applies to all ###
tfidf_x_text = x_text.apply(lambda x: re.sub('[^\w\s]', '', x.lower())) #To lowercase all text and remove punctuations
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=600) #Calling the Tfidf vectorizer object
tfidf_x_text = vectorizer.fit_transform(tfidf_x_text).toarray() #Convert the text data to numerical data

### With absolute metrics ###
tfidf_X_abs = np.concatenate((tfidf_x_text, x_metrics.values), axis=1) #Concate the converted text data with the metrics data
tfidf_X_train_abs, tfidf_X_test_abs, y_train_abs, y_test_abs = train_test_split(tfidf_X_abs, y_labels, test_size=0.2, random_state=3407) #Split the data into a training set and a test set

### With metrics delta ###
tfidf_X_delta = np.concatenate((tfidf_x_text, x_metrics_delta.values), axis=1)
tfidf_X_train_delta, tfidf_X_test_delta, y_train_delta, y_test_delta = train_test_split(tfidf_X_delta, y_labels, test_size=0.2, random_state=3407) #Split the data into a training set and a test set

### With all metrics ###
tfidf_X_all = np.concatenate((tfidf_x_text, x_metrics_all.values), axis=1) #Concate the converted text data with the metrics data
tfidf_X_train_all, tfidf_X_test_all, y_train_all, y_test_all = train_test_split(tfidf_X_all, y_labels, test_size=0.2, random_state=3407) #Split the data into a training set and a test set

In [35]:
### Understanding the features ###
print(f"There are {len(tfidf_X_train_abs[0])} features in the train dataset for absolute metrics only.")
print(f"There are {len(tfidf_X_train_delta[0])} features in the train dataset for metrics delta only.")
print(f"There are {len(tfidf_X_train_all[0])} features in the train dataset for all metrics.")

There are 604 features in the train dataset for absolute metrics only.
There are 604 features in the train dataset for metrics delta only.
There are 608 features in the train dataset for all metrics.


**Using the Decision Tree model**

In [36]:
### Absolute metrics only ###
DT_model_tfidf.fit(tfidf_X_train_abs, y_train_abs)
DT_y_pred_abs = DT_model_tfidf.predict(tfidf_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(y_test_abs, DT_y_pred_abs)}")

### Metrics delta only ###
DT_model_tfidf_delta.fit(tfidf_X_train_delta, y_train_delta)
DT_y_pred_delta = DT_model_tfidf_delta.predict(tfidf_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(y_test_delta, DT_y_pred_delta)}")

### All metrics ###
DT_model_tfidf_all.fit(tfidf_X_train_all, y_train_all)
DT_y_pred_all = DT_model_tfidf_all.predict(tfidf_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(y_test_all, DT_y_pred_all)}")

Accuracy for absolute metrics only: 0.47619047619047616
Accuracy for metrics delta only: 0.47619047619047616
Accuracy for all metrics: 0.5238095238095238


**Using the RandomForest model**

In [37]:
### Absolute metrics only ###
RF_model_tfidf.fit(tfidf_X_train_abs, y_train_abs)
RF_y_pred_abs = RF_model_tfidf.predict(tfidf_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(y_test_abs, RF_y_pred_abs)}")

### Metrics delta only ###
RF_model_tfidf_delta.fit(tfidf_X_train_delta, y_train_delta)
RF_y_pred_delta = RF_model_tfidf_delta.predict(tfidf_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(y_test_delta, RF_y_pred_delta)}")

### All metrics ###
RF_model_tfidf_all.fit(tfidf_X_train_all, y_train_all)
RF_y_pred_all = RF_model_tfidf_all.predict(tfidf_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(y_test_all, RF_y_pred_all)}")

Accuracy for absolute metrics only: 0.6428571428571429
Accuracy for metrics delta only: 0.6190476190476191
Accuracy for all metrics: 0.6428571428571429


**Using the XGBoost model**

In [38]:
### Absolute metrics only ###
XGB_model_tfidf.fit(tfidf_X_train_abs, y_train_abs)
XGB_y_pred_abs = XGB_model_tfidf.predict(tfidf_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(y_test_abs, XGB_y_pred_abs)}")

### Metrics delta only ###
XGB_model_tfidf_delta.fit(tfidf_X_train_delta, y_train_delta)
XGB_y_pred_delta = XGB_model_tfidf_delta.predict(tfidf_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(y_test_delta, XGB_y_pred_delta)}")

## All metrics ###
XGB_model_tfidf_all.fit(tfidf_X_train_all, y_train_all)
XGB_y_pred_all = XGB_model_tfidf_all.predict(tfidf_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(y_test_all, XGB_y_pred_all)}")

Accuracy for absolute metrics only: 0.5952380952380952
Accuracy for metrics delta only: 0.5714285714285714
Accuracy for all metrics: 0.5952380952380952


# Previously, we tried using a simple Tfidf vectorizer, here we will use the more advanced Glove word embedding

GloVe (Global Vectors for Word Representation) is a word embedding technique that represents words as dense vectors in a high-dimensional space. GloVe embeddings capture semantic and syntactic relationships between words based on their co-occurrence statistics in a corpus.

In [8]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize

In [40]:
### Initialize only for absolute metrics ###
DT_model_glove = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_glove = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_glove = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

### Initialize only for metrics delta ###
DT_model_glove_delta = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_glove_delta = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_glove_delta = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

### Initialize only for all metrics ###
DT_model_glove_all = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_glove_all = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_glove_all = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

In [ ]:
### To uncomment below the first time you run this to download the file from Stanford ###
!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
# Convert the GloVe file to word2vec format
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [43]:
# Load the converted GloVe vectors
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

def get_vector(word):
    if word in glove_model:
        return glove_model[word]
    else:
        return np.zeros(glove_model.vector_size)

In [44]:
glove_combined_df = combined_df.copy()
glove_combined_df['Statement'] = glove_combined_df['Statement'].apply(word_tokenize) #Tokenize the text
glove_combined_df['Vector'] = glove_combined_df['Statement'].apply(lambda x: np.mean([get_vector(w) for w in x], axis=0)) #Convert the text to vectors
glove_temp = np.array(glove_combined_df['Vector'].to_list())

In [45]:
### With absolute metrics only ###
glove_X_abs = np.concatenate((glove_temp, x_metrics.values), axis=1) #Concate the converted text data with the metrics data
glove_X_train_abs, glove_X_test_abs, glove_y_train_abs, glove_y_test_abs = train_test_split(glove_X_abs, y_labels, test_size=0.2, random_state=3407) #Split the data into a training set and a test set

### With delta metrics only ###
glove_X_delta = np.concatenate((glove_temp, x_metrics_delta.values), axis=1)
glove_X_train_delta, glove_X_test_delta, glove_y_train_delta, glove_y_test_delta = train_test_split(glove_X_delta, y_labels, test_size=0.2, random_state=3407)

### With all metrics
glove_X_all = np.concatenate((glove_temp, x_metrics_all.values), axis=1)
glove_X_train_all, glove_X_test_all, glove_y_train_all, glove_y_test_all = train_test_split(glove_X_all, y_labels, test_size=0.2, random_state=3407)

In [46]:
### Understanding the features ###
print(f"There are {len(glove_X_train_abs[0])} features in the train dataset for absolute metrics only.")
print(f"There are {len(glove_X_train_delta[0])} features in the train dataset for metrics delta only.")
print(f"There are {len(glove_X_train_all[0])} features in the train dataset for all metrics.")

There are 104 features in the train dataset for absolute metrics only.
There are 104 features in the train dataset for metrics delta only.
There are 108 features in the train dataset for all metrics.


**Using the Decision Tree model**

In [48]:
### Absolute metrics only ###
DT_model_glove.fit(glove_X_train_abs, glove_y_train_abs)
DT_y_pred_glove_abs = DT_model_glove.predict(glove_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(glove_y_test_abs, DT_y_pred_glove_abs)}")

### Metrics delta only ###
DT_model_glove_delta.fit(glove_X_train_delta, glove_y_train_delta)
DT_y_pred_glove_delta = DT_model_glove_delta.predict(glove_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(glove_y_test_delta, DT_y_pred_glove_delta)}")

### All metrics ###
DT_model_glove_all.fit(glove_X_train_all, glove_y_train_all)
DT_y_pred_glove_all = DT_model_glove_all.predict(glove_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(glove_y_test_all, DT_y_pred_glove_all)}")

Accuracy for absolute metrics only: 0.47619047619047616
Accuracy for metrics delta only: 0.5476190476190477
Accuracy for all metrics: 0.5714285714285714


**Using the Random Forest model**

In [50]:
### Absolute metrics only ###
RF_model_glove.fit(glove_X_train_abs, glove_y_train_abs)
RF_y_pred_glove_abs = RF_model_glove.predict(glove_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(glove_y_test_abs, RF_y_pred_glove_abs)}")

### Metrics delta only ###
RF_model_glove_delta.fit(glove_X_train_delta, glove_y_train_delta)
RF_y_pred_glove_delta = RF_model_glove_delta.predict(glove_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(glove_y_test_delta, RF_y_pred_glove_delta)}")

### All metrics ###
RF_model_glove_all.fit(glove_X_train_all, glove_y_train_all)
RF_y_pred_glove_all = RF_model_glove_all.predict(glove_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(glove_y_test_all, RF_y_pred_glove_all)}")

Accuracy for absolute metrics only: 0.5476190476190477
Accuracy for metrics delta only: 0.6190476190476191
Accuracy for all metrics: 0.6904761904761905


**Using the XGBoost model**

In [57]:
### Absolute metrics only ###
XGB_model_glove.fit(glove_X_train_abs, glove_y_train_abs)
XGB_y_pred_glove_abs = XGB_model_glove.predict(glove_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(glove_y_test_abs.values, XGB_y_pred_glove_abs)}")

### Metrics delta only ###
XGB_model_glove_delta.fit(glove_X_train_delta, glove_y_train_delta)
XGB_y_pred_glove_delta = XGB_model_glove_delta.predict(glove_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(glove_y_test_delta.values, XGB_y_pred_glove_delta)}")

### All metrics ###
XGB_model_glove_all.fit(glove_X_train_all, glove_y_train_all)
XGB_y_pred_glove_all = XGB_model_glove_all.predict(glove_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(glove_y_test_all.values, XGB_y_pred_glove_all)}")

Accuracy for absolute metrics only: 0.6190476190476191
Accuracy for metrics delta only: 0.5714285714285714
Accuracy for all metrics: 0.5714285714285714


# In this section, we will try the Doc2Vec embedding model.

It can be a good choice if the meaning of the statement is derived from the sequence of words or the overall context rather than the presence of individual words.

In [5]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [6]:
### Initialize only for absolute metrics ###
DT_model_D2V_abs = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_D2V_abs = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_D2V_abs = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

### Initialize only for metrics delta ###
DT_model_D2V_delta = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_D2V_delta = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_D2V_delta = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

### Initialize only for all metrics ###
DT_model_D2V_all = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_D2V_all = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_D2V_all = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

In [9]:
### Represent each document as a TaggedDocument object
tagged_documents = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(combined_df['Statement'])] 

In [10]:
### Train a Doc2Vec model on the Tagged docmuments ###
D2V_model = Doc2Vec(tagged_documents, vector_size=100, window=2, min_count=1, workers=4)

In [11]:
### Transform each document into a vector using the trained D2V_model ###
combined_df_D2V = combined_df.copy() 
combined_df_D2V['doc_vector'] = combined_df_D2V['Statement'].apply(lambda x: D2V_model.infer_vector(word_tokenize(x.lower())))

D2V_temp = np.array(combined_df_D2V['doc_vector'].to_list())

In [12]:
### With absolute metrics only ###
D2V_X_abs = np.concatenate((D2V_temp, x_metrics.values), axis=1) #Concate the converted text data with the metrics data
D2V_X_train_abs, D2V_X_test_abs, D2V_y_train_abs, DV2_y_test_abs = train_test_split(D2V_X_abs, y_labels, test_size=0.2, random_state=3407) #Split the data into a training set and a test set

### With metrics delta only ###
D2V_X_delta = np.concatenate((D2V_temp, x_metrics_delta.values), axis=1) #Concate the converted text data with the metrics data
D2V_X_train_delta, D2V_X_test_delta, D2V_y_train_delta, DV2_y_test_delta = train_test_split(D2V_X_delta, y_labels, test_size=0.2, random_state=3407) #Split the data into a training set and a test set

### With all metrics ###
D2V_X_all = np.concatenate((D2V_temp, x_metrics_all.values), axis=1) #Concate the converted text data with the metrics data
D2V_X_train_all, D2V_X_test_all, D2V_y_train_all, DV2_y_test_all = train_test_split(D2V_X_all, y_labels, test_size=0.2, random_state=3407) #Split the data into a training set and a test set

In [13]:
### Understanding the features ###
print(f"There are {len(D2V_X_train_abs[0])} features in the train dataset for absolute metrics only.")
print(f"There are {len(D2V_X_train_delta[0])} features in the train dataset for metrics delta only.")
print(f"There are {len(D2V_X_train_all[0])} features in the train dataset for all metrics.")

There are 104 features in the train dataset for absolute metrics only.
There are 104 features in the train dataset for metrics delta only.
There are 108 features in the train dataset for all metrics.



**Using the Decision Tree model**

In [15]:
### Absolute metrics only ###
DT_model_D2V_abs.fit(D2V_X_train_abs, D2V_y_train_abs)
DT_y_pred_D2V_abs = DT_model_D2V_abs.predict(D2V_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(DV2_y_test_abs, DT_y_pred_D2V_abs)}")

### Metrics delta only ###
DT_model_D2V_delta.fit(D2V_X_train_delta, D2V_y_train_delta)
DT_y_pred_D2V_delta = DT_model_D2V_delta.predict(D2V_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(DV2_y_test_delta, DT_y_pred_D2V_delta)}")

### All metrics ###
DT_model_D2V_all.fit(D2V_X_train_all, D2V_y_train_all)
DT_y_pred_D2V_all = DT_model_D2V_all.predict(D2V_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(DV2_y_test_all, DT_y_pred_D2V_all)}")

Accuracy for absolute metrics only: 0.6428571428571429
Accuracy for metrics delta only: 0.5476190476190477
Accuracy for all metrics: 0.5476190476190477


**Using the Random Forest model**

In [16]:
### Absolute metrics only ###
RF_model_D2V_abs.fit(D2V_X_train_abs, D2V_y_train_abs)
DT_y_pred_D2V_abs = RF_model_D2V_abs.predict(D2V_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(DV2_y_test_abs, DT_y_pred_D2V_abs)}")

### Metrics delta only ###
RF_model_D2V_delta.fit(D2V_X_train_delta, D2V_y_train_delta)
RF_y_pred_D2V_delta = RF_model_D2V_delta.predict(D2V_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(DV2_y_test_delta, RF_y_pred_D2V_delta)}")

### All metrics ###
RF_model_D2V_all.fit(D2V_X_train_all, D2V_y_train_all)
RF_y_pred_D2V_all = RF_model_D2V_all.predict(D2V_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(DV2_y_test_all, RF_y_pred_D2V_all)}")

Accuracy for absolute metrics only: 0.5714285714285714
Accuracy for metrics delta only: 0.5952380952380952
Accuracy for all metrics: 0.5952380952380952


**Using the XGBoost model**

In [17]:
### Absolute metrics only ###
XGB_model_D2V_abs.fit(D2V_X_train_abs, D2V_y_train_abs)
XGB_y_pred_D2V_abs = XGB_model_D2V_abs.predict(D2V_X_test_abs)
print(f"Accuracy for absolute metrics only: {accuracy_score(DV2_y_test_abs, XGB_y_pred_D2V_abs)}")

### Metrics delta only ###
XGB_model_D2V_delta.fit(D2V_X_train_delta, D2V_y_train_delta)
XGB_y_pred_D2V_delta = XGB_model_D2V_delta.predict(D2V_X_test_delta)
print(f"Accuracy for metrics delta only: {accuracy_score(DV2_y_test_delta, XGB_y_pred_D2V_delta)}")

### All metrics ###
XGB_model_D2V_all.fit(D2V_X_train_all, D2V_y_train_all)
XGB_y_pred_D2V_all = XGB_model_D2V_all.predict(D2V_X_test_all)
print(f"Accuracy for all metrics: {accuracy_score(DV2_y_test_all, XGB_y_pred_D2V_all)}")

Accuracy for absolute metrics only: 0.5952380952380952
Accuracy for metrics delta only: 0.6190476190476191
Accuracy for all metrics: 0.5952380952380952


# We will try using a pre-trained BERT model to extract features from the text data

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.9 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X_text_BERT = combined_df['Statement'].values #Get the text data into a np array
X_quant_BERT = combined_df[['Fed_fund_rate', 'Unemployment_rate', 'Core_CPI', 'US_10-2_Spread']].values #Get the numeric data into a np array

### Initiate a transformer model for feature extraction
feature_extraction = pipeline('feature-extraction', model='bert-base-uncased', tokenizer='bert-base-uncased', device=0, truncation=True, max_length=512) # device=0 to run on GPU
X_text_BERT_embeddings = np.array([np.mean(feature_extraction(text), axis=1)[0] for text in X_text_BERT])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
# Combine the text embedding with the metrics data #
X_BERT = np.concatenate((X_text_BERT_embeddings, X_quant_BERT), axis=1)
y_BERT = combined_df['Label'].values

### Split the data into training and test set ###
X_train_BERT, X_test_BERT, y_train_BERT, y_test_BERT = train_test_split(X_BERT, y_BERT, test_size=0.2, random_state=3407)

In [ ]:
### Understanding the features ###
print(f"There are {len(X_train_BERT[0])} features in the train dataset.")

There are 772 features in the train dataset.


In [ ]:
DT_model_BERT = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=3407))
RF_model_BERT = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=3407))
XGB_model_BERT = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=3407))

**Using the Decision Tree model**

In [ ]:
DT_model_BERT.fit(X_train_BERT, y_train_BERT)
DT_y_pred_BERT = DT_model_BERT.predict(X_test_BERT)
print(f"Accuracy: {accuracy_score(y_test_BERT, DT_y_pred_BERT)}")

Accuracy: 0.5


**Using the Random Forest model**

In [ ]:
RF_model_BERT.fit(X_train_BERT, y_train_BERT)
RF_y_pred_BERT = RF_model_BERT.predict(X_test_BERT)
print(f"Accuracy: {accuracy_score(y_test_BERT, RF_y_pred_BERT)}")

Accuracy: 0.5714285714285714


**Using the XGBoost model**

In [ ]:
XGB_model_BERT.fit(X_train_BERT, y_train_BERT)
XGB_y_pred_BERT = XGB_model_BERT.predict(X_test_BERT)
print(f"Accuracy: {accuracy_score(y_test_BERT, XGB_y_pred_BERT)}")

Accuracy: 0.5476190476190477


# We will now use the text embedding from OpenAI

In [ ]:
!pip install openai

In [ ]:
import openai 
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = "KEY"
openai.api_key = "KEY"

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [ ]:
fomc_statements_GPT = fomc_statements.copy()

In [ ]:
fomc_statements_GPT.head()

,Meeting_Date,Statement
0,1997-03-25,_x000D_\n_x000D_\n\tThe Federal Open Market Co...
1,1998-09-29,_x000D_\nThe Federal Open Market Committee dec...
2,1998-10-15,_x000D_\nThe Federal Reserve today announced t...
3,1998-11-17,_x000D_\nThe Federal Reserve today announced t...
4,1999-05-18,_x000D_\nThe Federal Reserve released the foll...


In [ ]:
fomc_statements_GPT['ada_embedding'] = fomc_statements_GPT.Statement.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [ ]:
X_text_GPT = np.array(fomc_statements_GPT['ada_embedding'].to_list())
X_quant_GPT = combined_df[['Fed_fund_rate', 'Unemployment_rate', 'Core_CPI', 'US_10-2_Spread']].values #Get the numeric data into a np array

# Combine the text embedding with the metrics data #
X_GPT = np.concatenate((X_text_GPT, X_quant_GPT), axis=1)
y_GPT = combined_df['Label'].values

### Split the data into training and test set ###
X_train_GPT, X_test_GPT, y_train_GPT, y_test_GPT = train_test_split(X_GPT, y_GPT, test_size=0.2, random_state=42)

In [ ]:
### Understanding the features ###
print(f"There are {len(X_train_GPT[0])} features in the train dataset.") #Seems like there are way too many parameters for the DTs to handle

There are 1540 features in the train dataset.


In [ ]:
DT_model_GPT = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=42))
RF_model_GPT = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42))
XGB_model_GPT = make_pipeline(StandardScaler(), xgb.XGBClassifier(objective='binary:logistic', random_state=42))

**Using the Decision Tree model**

In [ ]:
DT_model_GPT.fit(X_train_GPT, y_train_GPT)
DT_y_pred_GPT = DT_model_GPT.predict(X_test_GPT)
print(f"Accuracy: {accuracy_score(y_test_GPT, DT_y_pred_GPT)}")

Accuracy: 0.30952380952380953


**Using the Random Forest model**

In [ ]:
RF_model_GPT.fit(X_train_GPT, y_train_GPT)
RF_y_pred_GPT = RF_model_GPT.predict(X_test_GPT)
print(f"Accuracy: {accuracy_score(y_test_GPT, RF_y_pred_GPT)}")

Accuracy: 0.5


**Using the XGBoost model**

In [ ]:
XGB_model_GPT.fit(X_train_GPT, y_train_GPT)
XGB_y_pred_GPT = XGB_model_GPT.predict(X_test_GPT)
print(f"Accuracy: {accuracy_score(y_test_GPT, XGB_y_pred_GPT)}")

Accuracy: 0.4523809523809524


# We replicate all of the above but use delta values rather than absolute in the econometrics dataset